In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
# import tflearn
import tensorflow
import random

import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# **Loading our file and extracting the required information**

In [4]:
chat_path = '/content/drive/My Drive/intents.json'

import json
with open(chat_path) as file:
    data = json.load(file)

In [5]:
words = []
labels = []
docs_x = []
docs_y = []

In [6]:
print( data['intents'][0]['patterns'] )
print( data['intents'][0]['tag'] )

['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up']
greeting


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [9]:
print( 'list of all words in our file ' , words )

list of all words in our file  ['Hi', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Whats', 'up', 'cya', 'See', 'you', 'later', 'Goodbye', 'I', 'am', 'Leaving', 'Have', 'a', 'Good', 'day', 'how', 'old', 'how', 'old', 'is', 'tim', 'what', 'is', 'your', 'age', 'how', 'old', 'are', 'you', 'age', '?', 'what', 'is', 'your', 'name', 'what', 'should', 'I', 'call', 'you', 'whats', 'your', 'name', '?', 'Id', 'like', 'to', 'buy', 'something', 'whats', 'on', 'the', 'menu', 'what', 'do', 'you', 'reccommend', '?', 'could', 'i', 'get', 'something', 'to', 'eat', 'when', 'are', 'you', 'guys', 'open', 'what', 'are', 'your', 'hours', 'hours', 'of', 'operation']


In [10]:
print( 'words and their corresponding labels : ', list( zip( docs_x , docs_y ) ) )

words and their corresponding labels :  [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Whats', 'up'], 'greeting'), (['cya'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['how', 'old'], 'age'), (['how', 'old', 'is', 'tim'], 'age'), (['what', 'is', 'your', 'age'], 'age'), (['how', 'old', 'are', 'you'], 'age'), (['age', '?'], 'age'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'I', 'call', 'you'], 'name'), (['whats', 'your', 'name', '?'], 'name'), (['Id', 'like', 'to', 'buy', 'something'], 'shop'), (['whats', 'on', 'the', 'menu'], 'shop'), (['what', 'do', 'you', 'reccommend', '?'], 'shop'), (['could', 'i', 'get', 'something', 'to', 'eat'], 'shop'), (['when', 'are', 'you', 'guys', 'open'], 'hours'), (['what', 'are', 'your', 'hours'], 'hours'), (

In [11]:
print( 'Labels in our text : ', labels )

Labels in our text :  ['greeting', 'goodbye', 'age', 'name', 'shop', 'hours']


# **Preprocessing our data for model building**

In [9]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [10]:
labels

['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']

In [11]:
# BOW convertion for model building 

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [15]:
print( 'Sample of how the data is converted..')
print()
print( 'Input :',docs_x[0] )
print( 'Output label :',docs_y[0] )
print( 'Corresponding word : ' , words )
print( 'Corresponding labels : ' , labels )
print()
print( 'Transformed Input : ', training[0] )
print( 'Transformed Output : ', output[0] )

Sample of how the data is converted..

Input : ['Hi']
Output label : greeting
Corresponding word :  ['a', 'ag', 'am', 'anyon', 'ar', 'buy', 'cal', 'could', 'cya', 'day', 'do', 'eat', 'get', 'good', 'goodby', 'guy', 'hav', 'hello', 'hi', 'hour', 'how', 'i', 'id', 'is', 'lat', 'leav', 'lik', 'menu', 'nam', 'of', 'old', 'on', 'op', 'reccommend', 'see', 'should', 'someth', 'the', 'ther', 'tim', 'to', 'up', 'what', 'when', 'yo', 'you']
Corresponding labels :  ['age', 'goodbye', 'greeting', 'hours', 'name', 'shop']

Transformed Input :  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Transformed Output :  [0, 0, 1, 0, 0, 0]


# **Model Building**

In [12]:
import keras 
from keras.layers import Dense
from keras.models import Sequential 


Using TensorFlow backend.


In [13]:
 len(training[0])

46

In [14]:
model = Sequential()
model.add(  Dense( units = 8 , input_dim  =  len(training[0]) ) )
model.add(  Dense(8, activation="relu") )
model.add(  Dense(8, activation="relu") )
model.add(  Dense(len( output[0] ), activation="softmax" ) )
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 376       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 54        
Total params: 574
Trainable params: 574
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile( optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'] )

In [16]:
training = numpy.array( training )
output = numpy.array( output )

In [17]:
model.fit( training , output , epochs = 1000 )

Epoch 1/1000
26/26 [==============================] - 0s 11ms/step - loss: 0.4497 - accuracy: 0.8333
Epoch 2/1000
26/26 [==============================] - 0s 83us/step - loss: 0.4491 - accuracy: 0.8333
Epoch 3/1000
26/26 [==============================] - 0s 130us/step - loss: 0.4483 - accuracy: 0.8333
Epoch 4/1000
26/26 [==============================] - 0s 68us/step - loss: 0.4475 - accuracy: 0.8333
Epoch 5/1000
26/26 [==============================] - 0s 118us/step - loss: 0.4467 - accuracy: 0.8333
Epoch 6/1000
26/26 [==============================] - 0s 140us/step - loss: 0.4460 - accuracy: 0.8333
Epoch 7/1000
26/26 [==============================] - 0s 70us/step - loss: 0.4453 - accuracy: 0.8333
Epoch 8/1000
26/26 [==============================] - 0s 66us/step - loss: 0.4445 - accuracy: 0.8333
Epoch 9/1000
26/26 [==============================] - 0s 91us/step - loss: 0.4438 - accuracy: 0.8333
Epoch 10/1000
26/26 [==============================] - 0s 60us/step - loss: 0.4430 - acc

# **Saving and Loading our trained model**

In [24]:
model_json = model.to_json()

with open( 'model.json' , 'w' ) as json_file :
    json_file.write( model_json )
    
# serialize weight to HDF5

model.save_weights( 'model1.h5' )

print( "Saved model to disk ")

Saved model to disk 


In [25]:
from keras.models import model_from_json 

json_path = '/content/model.json'
weights = '/content/model1.h5'

with open( json_path , 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json( loaded_model_json)
# load weights into new model
loaded_model.load_weights(weights)

print( "Loaded model from desk" )

Loaded model from desk


# **Making Predictions**

In [28]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = list( set( [stemmer.stem(word.lower()) for word in s_words] ) )

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    arr = numpy.array( bag )
    arr = arr.reshape( 1 , -1 ) 
    return arr

In [29]:
import random

In [32]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You : ")
        if inp.lower() == "quit":
            break

        results = loaded_model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print('JJ Bot :' , random.choice(responses))

# **Let's chat with our BOT**

In [33]:
chat()

Start talking with the bot (type quit to stop)!
You : hi
JJ Bot : Good to see you again!
You : whats your name
JJ Bot : I'm JJ aka Jay.
You : age
JJ Bot : I am 18 years old!
You : whats on the menu
JJ Bot : Cookies are on the menu!
You : hour of working
JJ Bot : We are open 7am-4pm Monday-Friday!
You : see you later
JJ Bot : Talk to you later
You : quit
